<a href="https://colab.research.google.com/github/KaihoWakayama/LearnPytorch/blob/main/section_2/03_simple_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# シンプルなBERTの実装
訓練済みのモデルを使用し、文章の一部の予測、及び2つの文章が連続しているかどうかの判定を行います。

## ライブラリのインストール
PyTorch-Transformers、および必要なライブラリのインストールを行います。

In [1]:
!pip install folium==0.2.1
!pip install urllib3==1.25.11
!pip install pytorch-transformers==1.2.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.0/70.0 kB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for folium: filename=folium-0.2.1-py3-none-any.whl size=79793 sha256=e15f615e365ae85db249ea0ae34343b5ee197698e0049f6d81760865f94d90ba
  Stored in directory: /root/.cache/pip/wheels/91/87/f6/9abb612feb9dc3cdfd399a1ec49d0baa685596525ea0513d94
Successfully built folium
  Attempting uninstall: folium
    Found existing installation: folium 0.19.7
    Uninstalling folium-0.19.7:
      Successfully uninstalled folium-0.19.7
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.17.6 requires folium>=0.9.1, but you have folium 0.2.1 which is incompatible.
geemap 0.35.3 requires folium>=0.17.0, but you have folium 0.2.1 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.1/41.1 kB 2.1 MB/s eta 0:00:00
   ━━━

## 文章の一部の予測
文章における一部の単語をMASKし、それをBERTのモデルを使って予測します。

In [2]:
import torch
from pytorch_transformers import BertForMaskedLM
from pytorch_transformers import BertTokenizer


text = "[CLS] I played baseball with my friends at school yesterday.[SEP] It was really hard to hit his pitch."
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
words = tokenizer.tokenize(text)
print(words)

100%|██████████| 231508/231508 [00:00<00:00, 1305933.27B/s]


['[CLS]', 'i', 'played', 'baseball', 'with', 'my', 'friends', 'at', 'school', 'yesterday', '[SEP]']


文章の一部をMASKします。

In [3]:
msk_idx = 3
words[msk_idx] = "[MASK]"  # 単語を[MASK]に置き換える
print(words)

['[CLS]', 'i', 'played', '[MASK]', 'with', 'my', 'friends', 'at', 'school', 'yesterday', '[SEP]']


単語を対応するインデックスに変換します。

In [4]:
word_ids = tokenizer.convert_tokens_to_ids(words)  # 単語をインデックスに変換
word_tensor = torch.tensor([word_ids])  # テンソルに変換
print(word_tensor)

tensor([[ 101, 1045, 2209,  103, 2007, 2026, 2814, 2012, 2082, 7483,  102]])


BERTのモデルを使って予測を行います。

In [5]:
msk_model = BertForMaskedLM.from_pretrained("bert-base-uncased")
msk_model.cuda()  # GPU対応
msk_model.eval()

x = word_tensor.cuda()  # GPU対応
y = msk_model(x)  # 予測
result = y[0]
print(result.size())  # 結果の形状

_, max_ids = torch.topk(result[0][msk_idx], k=5)  # 最も大きい5つの値
result_words = tokenizer.convert_ids_to_tokens(max_ids.tolist())  # インデックスを単語に変換
print(result_words)

100%|██████████| 440473133/440473133 [00:08<00:00, 51030734.20B/s]


torch.Size([1, 11, 30522])
['basketball', 'football', 'soccer', 'baseball', 'tennis']


`y[0]` は、BERTモデルが予測した各トークン（単語や句読点など）に対する、すべての可能な単語（ボキャブラリ）についての確率分布のテンソルです。具体的には、入力された文章に含まれる各トークンに対して、その位置にどの単語が来る可能性が高いかを表しています。

`y` に含まれているその他の値（今回のコードでは `y[0]` 以外は直接使われていませんが、一般的に `y` はタスクによって異なる構造を持ち得ます）は、使用しているBERTモデルの種類（ここでは `BertForMaskedLM`）と、BERTモデルの出力によって決まります。

今回の `BertForMaskedLM` モデルの場合、出力 `y` はタプルの形式になっており、そのタプルの最初の要素 `y[0]` が、前述の確率分布のテンソルです。

まとめると：

*   **`y[0]`**: 入力された文章中の各トークンの位置における、全ボキャブラリに対する確率分布のテンソル。`result = y[0]` でこの部分を取り出し、maskedされた位置の単語予測に使用しています。
*   **その他yに含まれている値**: 今回の `BertForMaskedLM` の場合は、タプルの2番目以降の要素（例えば、アテンションウェイトや隠れ層の出力など）が含まれる可能性がありますが、このコードではそれらは使用されていません。一般的なBERTモデルの出力はタスク（例えば、次文予測など）によって構造が変わることがあります。

## 文章が連続しているかどうかの判定
BERTのモデルを使って、2つの文章が連続しているかどうかの判定を行います。  
以下の関数`show_continuity`では、2つの文章の連続性を判定し、表示します。

In [9]:
from pytorch_transformers import BertForNextSentencePrediction

def show_continuity(text, seg_ids):
    words = tokenizer.tokenize(text)
    word_ids = tokenizer.convert_tokens_to_ids(words)  # 単語をインデックスに変換
    word_tensor = torch.tensor([word_ids])  # テンソルに変換

    seg_tensor = torch.tensor([seg_ids])

    nsp_model = BertForNextSentencePrediction.from_pretrained('bert-base-uncased')
    nsp_model.cuda()  # GPU対応
    nsp_model.eval()

    x = word_tensor.cuda()  # GPU対応
    s = seg_tensor.cuda()  # GPU対応

    y = nsp_model(x, s)  # 予測
    result = torch.softmax(y[0], dim=1)
    print(result)  # Softmaxで確率に
    print(str(result[0][0].item()*100) + "%の確率で連続しています。")

`show_continuity`関数に、自然につながる2つの文章を与えます。

In [ ]:
text = "[CLS] What is baseball ? [SEP] It is a game of hitting the ball with the bat [SEP]"
seg_ids = [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1 ,1, 1]  # 0:前の文章の単語、1:後の文章の単語
show_continuity(text, seg_ids)

`show_continuity`関数に、自然につながらない2つの文章を与えます。

In [ ]:
text = "[CLS] What is baseball ? [SEP] This food is made with flour and milk [SEP]"
seg_ids = [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1]  # 0:前の文章の単語、1:後の文章の単語
show_continuity(text, seg_ids)